In [98]:
using JuMP
using SumOfSquares
using MosekTools
using DynamicPolynomials
using LinearAlgebra

## Example 1 
$\newcommand\Re{\mathbb{R}}$
Find a polynomial $p  \in \Re[x,y]$ such that 
   - $p(\vec{0}) = 0$.
   - p(1, 1) = 1
   - $p$ is SOS.
   - $(x^2 + y^2 \leq 1)\ \models\ p \leq 1$ 

In [115]:
function sos_example_1a()
    @polyvar x y 

    #initialize a solver object 
    solver = optimizer_with_attributes(MosekTools.Optimizer)
    # create a model for sum of squares optimization 
    model = SOSModel(solver)
    # create a list of monomials of degrees 1 and 2
    # since p(0) = 0, we simply omit the constant term from p
    monos = monomials([x,y], 1:2)
    n_monos = size(monos)[1]
    @variable(model, A[1:n_monos])
    p = A ⋅ monos 
    println("p = $p")
    # specify that p is some unknown polynomial involving monos 
    println("p(1,1) = $(p(x => 1, y => 1))")
    @constraint(model, c0, p(x => 1, y => 1) == 1)
    S = @set(1 - x^2 - y^2 >= 0)
    @constraint(model, c1, p >= 0 ) # p must be positive definite
    @constraint(model, c2, 1 - p >= 0, domain=S) # specify that S ⊧ 1 - p >= 0
    @objective(model, Max, A[5] + A[3])
    optimize!(model)
    println(primal_status(model))
    println("Found p = $(value(p))")
    
    return value(p)
end

sos_example_1a (generic function with 1 method)

In [116]:
sos_example_1a()

p = (A[1])*y + (A[2])*x + (A[3])*y^2 + (A[4])*x*y + (A[5])*x^2
p(1,1) = A[1] + A[2] + A[3] + A[4] + A[5]
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 12              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 1               
  Scalar variables       : 9               
  Matrix variables       : 1 (scalarized: 6)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1   

0.7499999999998024y² - 0.4999999999994705xy + 0.7499999999998029x²

# Example 2

We wish to prove the following entailment for the smallest possible $\alpha$

$$ x^2 + y^2 + z^2 \leq 1\ \land\  xy \geq \frac{1}{2} \ \land\ x + z \geq 0.1\ \models\ y \leq \alpha $$  

In [86]:
function sos_example_2()
    # declare x, y, z as variables
    @polyvar x y z
    p1 = 1 - x^2 - y^2 - z^2
    p2 = x*y - 0.5
    p3 =  x+ z -0.1
    # initialize the LHS of the entailment
    S = @set(p1 >= 0) ∩ @set(p2 >= 0) ∩ @set(p3 >= 0)
    
    solver = optimizer_with_attributes(MosekTools.Optimizer)
    model = SOSModel(solver)
    @variable(model, α)
    @objective(model, Min, α)
    @constraint(model, c1, α - y >= 0, domain = S)
    optimize!(model)
    println(primal_status(model))
    println("alpha = $(value(α))")
    #println(lagrangian_multipliers(c1))
end

sos_example_2 (generic function with 1 method)

In [85]:
# this takes a while to run first time around due to pre-compilation overhead.
sos_example_1()

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 4               
  Matrix variables       : 1 (scalarized: 10)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep.  - dual attempts          : 0         